# 🌐 环境与作用域 (Environment & Scope) 核心笔记

## 📚 知识点概览

| 概念 | 关键词 | 核心作用 |
| :--- | :--- | :--- |
| **环境图 (Environment Diagram)** | 定义时/执行时, 父框架, 局部框架 | 追踪变量绑定和作用域（词法作用域）的工具。 |
| **闭包 (Closure)** | 嵌套函数, 词法作用域, 记住环境 | 内部函数访问并保存外部函数的局部变量。 |
| **高阶函数 (HOFs)** | 函数作为参数, 函数作为返回值 | 实现抽象和柯里化。 |
| **Lambda 表达式** | 匿名函数, 单一表达式, 无内在名称 | 快速创建简洁的函数值。 |

---

## Ⅰ. 环境图绘制规则 (The Environment Model)

### 1. 函数定义时 (Definition Time)

> [!INFO] **函数定义时的处理**
> **创建函数值：** 生成函数对象 `func <name>(<parameters>) [parent=<parent>]`。
> **父框架引用 (Parent Frame)：** 函数对象**记住**了它被定义时的环境。这是**词法作用域 (Lexical Scoping)**的基础。
> **名称绑定：** 将函数名（或 Lambda 表达式）绑定到当前框架中的这个函数对象。
> **特性：** 定义时不执行函数体代码 (惰性求值)。

### 2. 函数执行时 (Execution Time)

> [!IMPORTANT] **函数调用的步骤**
> **创建局部框架：** 生成一个新的局部框架，标题为被调用函数名。
> **设置父框架（关键）：** 新框架的父框架被设置为函数对象中记录的**父框架**（定义时的环境）。
> **参数绑定：** 将实参值绑定到局部框架中的形参名。
> **执行函数体：** 在新框架环境中开始执行代码。

---

## Ⅱ. 作用域与高阶函数应用

### 1. 词法作用域与非嵌套函数

> [!WARNING] **局部变量的隔离**
> * **问题：** 当非嵌套函数 `g` 被调用时，它无法访问另一个非嵌套函数 `f` 中的局部变量 `y`。
> * **原因：** `f` 和 `g` 都是在 `Global` 中定义的**兄弟框架**。`g` 的作用域链（`g` $\rightarrow$ `Global`）永远无法进入 `f` 的局部环境。
> * **结果：** 抛出 `NameError`。

### 2. 闭包与查找链

> [!EXAMPLE] **嵌套函数查找外部变量 (Closure)**
> ```python
> def make_adder(n):
>     def adder(k):
>         return n + k  # 查找 n
>     return adder
> # 调用：add_three = make_adder(3); add_three(4)
> ```
> * **`adder` 的父框架**：是 `make_adder(3)` 的局部框架（`f1`）。
> * **查找 `n`**：`adder(4)` 会沿着父框架链，回到 `f1` 框架中找到 `n=3`。

### 3. 函数作为参数 (HOF)

> [!IMPORTANT] **高阶函数机制**
> * **机制：** 函数值可以作为实参传递（例如 `apply_twice(square, 2)`）。
> * **环境图表现：** 在接收函数的局部框架中，对应的形参（如 `f`）会**直接绑定到传入的函数对象**。

### 4. 柯里化 (Currying)

> [!NOTE] **函数链式调用**
> * **概念：** 将一个多参数函数转换为一系列只接受**单参数**的函数。
> * **示例：** `make_adder(2)(3)`
> * **执行：** 分解了原本的加法操作，并利用闭包记住了第一个参数 `n=2`。

---

## Ⅲ. 函数类型与机制

### 1. Lambda 表达式 vs. Def 语句

> [!TIP] **Lambda 与 Def 的对比**
> * **功能行为：** 两者创建的函数在行为上是**等价**的，且父框架记录相同。
> * **内在名称：** `def` 赋予函数内在名称；`lambda` 创建的是**匿名函数**。
> * **限制：** `lambda` 主体必须是**单一表达式**，不能包含语句。

### 2. 并行赋值的机制

> [!NOTE] **`pred, curr = curr, curr + pred`**
> 1.  **求值右侧 (RHS)——原子性：** 解释器计算右侧所有表达式的值，并存储在一个**临时元组**中。
> 2.  **同时赋值 (LHS)：** 将临时元组中的值同时赋给左侧的变量。
> * **特性：** 保证右侧的计算总是使用赋值操作**之前**的旧值。